In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ipywidgets import *
from IPython.display import display

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<img src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" style="float: left; margin: 15px;">
### Introduction to SVM
Week **5** | Lesson **6 **


### Lesson Objectives

*After this lesson, you will be able to:*
- **Demenstrate** an understanding of how SVM separates data with a plane with written responses and graphics 
- **Analyze** and understand how the hyperparameter $C$ affects SVM's soft margins 
- **Explain** how the hinge loss error function works 
- **Describe** what the kernal trick is and how it works in terms of linear algebra and graphics

---
### Student Pre-Work 

*Before this lesson, you should already be able to:*
- Explian what a linear transformation is in terms of linear algebra

----

## Lesson

The Support Vector Machine (SVM) algorithm is a different approach to classification.

These fit a decision boundary similarly to a logistic regression, but uses a different loss function called the "hinge loss" (as opposed to the log loss in logistic regression).

SVMs are notorious for being less intuitive than other classifiers like kNN and Logistic regression, but are quite powerful

[For a really great resource check out these slides (some of which are cannabalized in this lecture).](http://www.robots.ox.ac.uk/~az/lectures/ml/lect2.pdf)

[This website is also a great resource, on a slightly more technical level.](http://nlp.stanford.edu/IR-book/html/htmledition/support-vector-machines-the-linearly-separable-case-1.html)


## How does the SVM classify?

---

It's important to start with the intuition for the special _**linearly separable**_ classification case.


If classification of observations is "linearly separable", SVM fits the **"decision boundary"** that is defined by the largest margin between the closest points for each class. This is commonly called the **"maximum margin hyperplane (MMH)"**.

![linearly separable SVM](./images/linear_separability_vs_not.png)

## Intuition behind the SVM decision boundary

---

SVM's criterion for a decision surface is one that is _maximally far away from any data point between classes_. The distance from the decision boundary to the closest data point determines the "margin" of the classifier.

The points SVM uses to fit the decision boundary are called "support vectors". This term comes from linear algebra: in a vector space, points can be defined as a vector between the origin and that point. 

![linearly separable SVM MMH margin](./images/Margin.png)

## Why maximize the margin?

---

**SVM solves for a decision boundary that hypothetically minimizes generalization error.** 

Observations that are near the decision boundary between the classes are the most ambiguous observations. They are the observations that are approaching equal chance to be one class or the other.

SVM, instead of considering all the observations "equally" in the loss function it minimizes, defines it's fit using the most ambiguous points. It's decision boundary is _safe_ in that errors in new measured observations are not likely to cause the SVM to mis-classify.

The SVM is concerned with generalization to new data.

## The Perceptron Algorithm

---

The perceptron algorithm is a linear function of weights $w$ and predictors $X$ that assigns points into binary classes. If the function returns a value greater than 0 then the observation is classified as 1, otherwise it is classified as zero.

$f(X)$ below is the perceptron function that would determine the classes. $b$ here is known as the "bias" and is analagous to the intercept in a regression.

### $$ f(X) = \begin{cases}1 & \text{if }w \cdot x + b > 0\\0 & \text{otherwise}\end{cases} $$

If the points are linearly seperable, solving the perceptron is guaranteed to converge on a solution, but that solution is not necessarily optimal for future observations. This led to the invention of the SVM, which finds the optimal discriminator: the maximum margin hyperplane.




## The maximum margin hyperplane

---

We can impose a "normalizing constant" $c$ on the equation for the line or hyperplane that separates the points:

### $$ c(w^T x + b) = 0 $$

And then choose this normalizing constant such that the distance from the plane to the closest points of either class will be 1.

### $$ w^T x_{+} + b = 1 \\ w^T x_{-} + b = -1 $$

For the distance to the closest positive and negative class points, respectively (known as "support vectors").

If the normalizer for the weights is $||w||$, the size of the margin is then:

### $$ \text{margin} = \frac{2}{||w||} $$


![linearly separable SVM MMH margin](./images/linear_sep_support_vecs_math.png)

## Finding the maximum margin

---

We want to find a distance between these points that is the biggest possible. Therefore we are looking to maximize the value $\frac{2}{||w||}$.

So, maximize the weights with constraints on what the function can output. When the target is 1, the function needs to be 1 or greater. When the target is 0 (or -1), the function needs to output -1 or lower.

### $$ \underset{w}{\text{max}} \frac{2}{||w||} \quad \text{subject to} \begin{cases}\text{if } y_i = 1 & w^T x_i + b \ge 1 \\ \text{if } y_i = -1 & w^T x_i + b \le -1 \end{cases} \text{for } i \text{ in } N$$

Note here that $y$ is specified as either 1 or -1, as opposed to the 0, 1 that we are used to. This is convenient for converting this to be a minimization. To make this a minimization, we can change the equation to be:

### $$ \underset{w}{\text{min }} ||w||^2 \quad \text{subject to} \quad y(w^T x_i + b) \ge 1 $$

Which works out because when $y = -1$ the values become positive.

![linearly separable SVM MMH margin](./images/linear_sep_support_vecs.png)

## Non-linearly separable cases and the hinge loss function

---

In cases where there is no line or plane that can separate all of the points perfectly, we need to introduce the capability for error into our equation. With the constraint above that $y(w^T X + b) \ge 1$, we can introduce the **hinge loss function**:

### $$ \text{hinge loss} = \sum_{i=1}^n \max\left(0, 1 - y_i(w^T x_i + b)\right) $$

Where now, if the point is on the correct side (correctly classified), the value of the maximization will be 0. If the point is not $\ge 1$, this function will be greater than zero.

Using $f(x_i) = (w^T x_i + b)$, 

### $$\text{hinge loss} = \sum_{i=1}^N max\left(0, 1 - y_i \: f(x_i)\right)$$ 

If $f(x_i) > 1$, the point lies _outside_ the margin and does not contribute to the loss.

If $f(x_i) = 1$ the point is _on_ the margin and does not contribute to the loss.

If $f(x_i) < 1$ the point lies _inside_ the margin and **does** contribute to the loss.


![Hinge loss](./images/hinge_loss.png)

## Hinge loss and "slack"

---

When we have a scenario where it is not possible to perfectly separate, we use the hinge loss with a regularization constant $C$:

### $$ \min ||w||^2 + C\sum_{i=1}^N \epsilon_i \quad \text{subject to} \quad y(w^T x_i + b) \ge 1 - \epsilon_i $$

Where the $\epsilon$ are the errors of the classifier, and the $C$ is a regularization term that determines how much the classification errors matter (relative to the maximization of the margin).

The function that the SVM minimizes to find the boundary is:

### $$  \underset{w}{\text{min }} ||w||^2 + C\sum_{i=1}^N max\left(0, 1 - y_i(w^T x_i + b)\right) $$

A small $C$ creates a wider margin because errors will matter less. A large $C$ creates a tighter margin because errors matter more. An infinite $C$ parameter is a "hard" margin, which always minimizes error over the size of the boundary.

We are trying to minimize the norm of the weights $||w||$ and thus maximize the margin, but we are _also_ trying to minimize our errors at the same time. We have a balance in our minimization between how wide the margin should be and how much error we should incur.


## The regularizing hyper-parameter $C$

---

The hyper-parameter $C$ controls the extent to which the SVM is tolerant to errors. It is sometimes called the "soft-margin constant". 

$C$ affects the strength of the "penalty" similar to the lambda terms in the Ridge and Lasso. 

By multiplying the sum of the errors, which are the distances from the margins to the points inside of the margin, it allows the SVM to classify non-linearly separable problems by allowing errors to occur. 

The lower the value of $C$, the more misclassified observations errors are allowed. These misclassified points are known as "slack variables". Reducing the effect of errors on the loss function puts the emphasis on widening the margin.

For those interested in exporing the math more, [there is a good tutorial here.](http://www.svm-tutorial.com/2015/06/svm-understanding-math-part-3/#more-457)

![soft margin](./images/slack_variables.png)

![soft margin](./images/soft_margin.png)

### Intuit Soft Margins 
[Interactive SVM Widget](Selective-Vector-Machines/svm-visualization.ipynb) **Check out this notebook** for an interactive viz on the effects of increasing and decreasing the value of hyperparameter $C$

### Questions

1. What happens when you change the value of $C$ to 5? Is this an instance of overfitting or underfitting?

In [ ]:
# change this cell to markdown and write your answer 

2. What happens when you change the value of $C$ to -4? Is this an instance of overfitting or underfitting?

In [ ]:
# change this cell to markdown and write your answer 

## The "kernel trick" for non-linearly seperable problems

---

The "kernel trick" allows an SVM to classify non-linearly separable problems. It is a big reason why SVMs are so popular.

The idea behind the kernel trick is that you can arbitrarily transform your observations that _have no linear seperability_ by putting them into a different "dimensional space" where the DO have linear seperability, fit an SVM in that higher dimensional space, and then invert the transformation of the data and the model itself back into the original space.

This is done by "wrapping" your predictors in a kernel function that transforms them into this higher dimensional space. 

[Check out these lecture slides for more detail.](http://www.robots.ox.ac.uk/~az/lectures/ml/lect3.pdf)

The following pictures should give you a general intuition for what is happening.

![kernel transform viz](./images/kernel_trick.png)

### Anamation 
The animation shows how data from a 2 dimensional vector space can be transformed into 3 dimensional vector space. Once the data has been transformed into a higher dimensional space, a plane can freely pass between the blue and red data points and create a boundary. Once the plane is optimually fitted between the blue and red points, the data is transformed back into the lower 
2 dimensional vector space. 

![](http://rvlasveld.github.io/images/oc-svm/visualization.gif)

**Note:** The colors between the animation and the static pictures are reversed. 



![nonsep_transformed](images/Nonseparable_Transformed.png)

We can transform the data visualized in the above figure from $[x,y]$ space to $[x,y,x^2+y^2]$ space and suddenly, a clear boundary appears!

![twoD-to-threeD](images/TwoD_to_ThreeD_Boundary.png)

### Final Examples

The following images show how different kenerls fit to the same data set. For this data set, we can see that both the polynomial and gaussian kernels do a better job at creating a boundary between the classes than the linear kernel. 

![polynomial kernel](./images/nonlinear-1.png)

![gaussian kernel](./images/nonlinear-2.png)

----

## Conclusion

In this lesson we covered: 

- How SVM separates data with a plane with written responses and graphics 
- How the hyperparameter $C$ affects SVM's soft margins 
- How the hinge loss error function works 
- What the kernal trick is and how it works in terms of linear algebra and graphics

----

## Additional Resources 

[SVM Animation](https://www.youtube.com/watch?v=3liCbRZPrZA) A YouTube video showing how 2D data can be mapped into 3D with a gaussian kernal and how a plane can intersect the paraboloid to create a circular boundary between the two classes. 

[Optional: Deeper Dive into SVM Theory](./Optional-and-Supplemental-SVM-Lesson.ipynb) This jupyter notebook provides an exposer into the rigorous mathematics behind SVM. The lesson provided you just covered in class strikes a balance between math and concept in order to explain SVM as pain free as possible. The Deep Dive notebook will satisfy your curious about where the equations from today's lesson came from. 